In [1]:
import pandas as pd
import torch
import torch.nn as nn
from collections import defaultdict
from tqdm import tqdm
from torch.utils.data import DataLoader, Dataset
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

# Import learning rate scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import LightningModule
from sklearn.model_selection import KFold
import numpy as np

from transformers import MT5EncoderModel, MT5Tokenizer, MT5Config

# Create embedded data

In [2]:
embedder_config = MT5Config.from_pretrained("google/mt5-base")
embedder_tokenizer = MT5Tokenizer.from_pretrained("google/mt5-base")
embedder = MT5EncoderModel.from_pretrained("google/mt5-base", config=embedder_config)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'T5Tokenizer'. 
The class this function is called from is 'MT5Tokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.mt5.tokenization_mt5.MT5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [3]:
train_df = pd.read_csv("../data/train.csv", index_col="ID")
test_df = pd.read_csv("../data/test.csv", index_col="ID")

all_df = pd.concat([train_df, test_df], axis=0)

In [ ]:
embeddings_data = defaultdict(list)

BATCH_SIZE = 64
answers = all_df["answer"].tolist()
for i in tqdm(range(0, len(answers), BATCH_SIZE), desc="Processing batches"):

    batch = answers[i : i + BATCH_SIZE]
    tokenized_batch = embedder_tokenizer(batch, padding=True, truncation=True, return_tensors="pt").input_ids
    with torch.no_grad():
        embedded_answers = embedder(tokenized_batch).last_hidden_state[:, 0, :].numpy()
    for j, answer in enumerate(batch):
        embeddings_data[i + j] = torch.tensor(embedded_answers[j], dtype=torch.float32)

torch.save(embeddings_data, "../data/llm_embeds/mt5-answer-embeddings.pt")

Processing batches: 100%|██████████| 8/8 [00:58<00:00,  7.35s/it]


In [ ]:
embeddings_data = defaultdict(list)

for set_id in all_df["set"].unique():
    set_df = all_df[all_df["set"] == set_id]
    set_question = set_df["question"].iloc[0]
    tokenized_batch = embedder_tokenizer([set_question], padding=True, truncation=True, return_tensors="pt").input_ids
    with torch.no_grad():
        embedding = embedder(tokenized_batch).last_hidden_state[0, 0, :].numpy()
    embeddings_data[set_id] = torch.tensor(embedding, dtype=torch.float32)

torch.save(embeddings_data, "../data/llm_embeds/mt5-question-embeddings.pt")

In [6]:
del embedder

# Create a regressor

In [13]:
K_FOLDS = 10
BATCH_SIZE = 8

INPUT_SIZE = 768 * 2
LAYER_NUM = 4
DROPOUT_RATE = 0.2

LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-2

# Learning rate scheduler
LRS_PATIENCE = 3
LRS_FACTOR = 0.7

# Early stopping
ES_PATIENCE = 20

MAX_EPOCHS = 100

In [ ]:
# Load the data
train_df = pd.read_csv("../data/train.csv", index_col="ID")
question_embeddings = torch.load("../data/llm_embeds/mt5-question-embeddings.pt", weights_only=False)
answer_embeddings = torch.load("../data/llm_embeds/mt5-answer-embeddings.pt", weights_only=False)

In [15]:
# Assign fold to each row
train_df["fold"] = -1
for fold, (train_idx, val_idx) in enumerate(KFold(n_splits=K_FOLDS, shuffle=True, random_state=42).split(train_df)):
    train_df.loc[val_idx, "fold"] = fold

In [16]:
class MT5Dataset(Dataset):
    def __init__(
        self,
        df: pd.DataFrame,
        question_embeddings: dict[int, torch.Tensor],
        answer_embeddings: dict[str, torch.Tensor],
        has_score: bool = True,
    ) -> None:
        self.df = df
        self.question_embeddings = question_embeddings
        self.answer_embeddings = answer_embeddings

        self.has_score = has_score

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        # Get the embeddings
        question_embedding = self.question_embeddings[row["set"]]
        answer_embedding = self.answer_embeddings[idx]
        input_embedding = torch.cat([question_embedding, answer_embedding], dim=0).type(torch.float32)

        if self.has_score:
            return input_embedding, row["score"].astype(np.float32)
        else:
            return input_embedding

In [17]:
class MLPRegressor(LightningModule):
    def __init__(self, input_size: int, layer_num: int, dropout_rate: float) -> None:
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, input_size // 2),
            nn.LeakyReLU(),
            *[
                nn.Sequential(nn.Linear(input_size // 2, input_size // 2), nn.LeakyReLU(), nn.Dropout(dropout_rate))
                for _ in range(layer_num)
            ],
            nn.Linear(input_size // 2, 1),
            nn.Sigmoid(),
        )

        self.loss_fn = nn.MSELoss(reduction="mean")

    def forward(self, x):
        return self.model(x) * 5.0

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.loss_fn(y_hat, y)
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True)

        # Log the learning rate
        self.log(
            "learning_rate",
            self.trainer.optimizers[0].param_groups[0]["lr"],
            on_step=True,
            on_epoch=False,
            prog_bar=True,
        )

        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x).squeeze()
        loss = self.loss_fn(y_hat, y)
        self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        scheduler = ReduceLROnPlateau(optimizer, mode="min", factor=LRS_FACTOR, patience=LRS_PATIENCE, verbose=True)
        return {
            "optimizer": optimizer,
            "lr_scheduler": scheduler,
            "monitor": "val_loss",
        }

In [18]:
models = [MLPRegressor(input_size=INPUT_SIZE, layer_num=LAYER_NUM, dropout_rate=DROPOUT_RATE) for _ in range(K_FOLDS)]
trainers = []
for current_fold in range(K_FOLDS):
    train_fold_df = train_df[train_df["fold"] != current_fold]
    val_fold_df = train_df[train_df["fold"] == current_fold]

    # Create the datasets and dataloaders
    train_dataset = MT5Dataset(train_fold_df, question_embeddings, answer_embeddings)
    val_dataset = MT5Dataset(val_fold_df, question_embeddings, answer_embeddings)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)

    # Create logger
    logger = CSVLogger("logs/mt5", name=f"mt5-{current_fold}")

    # Create a checkpoint callback
    checkpoint_callback = ModelCheckpoint(
        monitor="val_loss",
        dirpath=f"./logs/mt5/mt5-{current_fold}/checkpoints",
        filename="mt5-{epoch:02d}-{val_loss:.2f}",
        save_top_k=2,
        mode="min",
    )

    # Create an early stopping callback
    early_stopping = EarlyStopping(monitor="val_loss", patience=ES_PATIENCE, mode="min")

    # Create a trainer
    trainer = Trainer(max_epochs=MAX_EPOCHS, logger=logger, callbacks=[checkpoint_callback, early_stopping])

    # Fit the model
    trainer.fit(models[current_fold], train_loader, val_loader)

    # Append the trainer
    trainers.append(trainer)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name    | Type       | Params | Mode 
-----------------------------------------------
0 | model   | Sequential | 3.5 M  | train
1 | loss_fn | MSELoss    | 0      | train
-----------------------------------------------
3.5 M     Trainable params
0         Non-trainable params
3.5 M     Total params
14.174    Total estimated model params size (MB)
22        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/jirayuwat/anaconda3/envs/nlp/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/jirayuwat/anaconda3/envs/nlp/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/Users/jirayuwat/anaconda3/envs/nlp/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:310: The number of training batches (41) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [19]:
# Load the best model
best_models = []
for current_fold in range(K_FOLDS):
    model = MLPRegressor.load_from_checkpoint(
        trainers[current_fold].checkpoint_callback.best_model_path,
        input_size=INPUT_SIZE,
        layer_num=LAYER_NUM,
        dropout_rate=DROPOUT_RATE,
    )
    best_models.append(model)

# Create submission

In [20]:
test_df = pd.read_csv("../data/test.csv", index_col="ID")

test_dataset = MT5Dataset(test_df, question_embeddings, answer_embeddings, has_score=False)
test_loader = DataLoader(test_dataset, batch_size=4)

predictions = []
for model in best_models:
    model.eval()
    model.freeze()

    model_predictions = []
    for x in test_loader:
        y_hat = model(x.to("mps")).squeeze()
        model_predictions.extend(y_hat.tolist())

    predictions.append(model_predictions)

# Average the predictions
predictions = np.mean(predictions, axis=0)

test_df["score"] = predictions
test_df[["score"]].to_csv("mt5_submission.csv")